## 데이터 불러오기

In [ ]:
import pandas as pd

DATA_PATH = './csv_data/nocolorinfo'

train_df = pd.read_csv(DATA_PATH + '/train.csv')
val_df = pd.read_csv(DATA_PATH + '/val.csv')
test_df = pd.read_csv(DATA_PATH + '/test.csv')

# Colab에서 사용한다면, 다음 코드 주석을 풀고, 실행시킵니다.
# 또한, Colab에서 사용하는 경우, file reading 속도가 매우 느릴 수 있다는 점에 주의하세요.
# train_df['image'] = train_df['image'].apply(lambda x: str(x).replace('\\', '/'))
# val_df['image'] = val_df['image'].apply(lambda x: str(x).replace('\\', '/'))
# test_df['image'] = test_df['image'].apply(lambda x: str(x).replace('\\', '/'))

train_df.head()

## 이미지 제네레이터 정의하기

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 이미지 제네레이터를 정의합니다.
train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)

def get_steps(num_samples, batch_size):
    if (num_samples % batch_size) > 0 :
        return (num_samples // batch_size) + 1
    else :
        return num_samples // batch_size

## 모델 구성하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

model = Sequential()

# 입력 데이터의 형태를 꼭 명시해야 합니다.
model.add(Flatten(input_shape = (112, 112, 3))) # (112, 112, 3) -> (112 * 112 * 3)
model.add(Dense(128, activation = 'relu')) # 128개의 출력을 가지는 Dense 층
model.add(Dense(64, activation = 'relu')) # 64개의 출력을 가지는 Dense 층
model.add(Dense(11, activation = 'sigmoid')) # 11개의 출력을 가지는 신경망

model.compile(optimizer = 'adam', 
              loss = 'binary_crossentropy', 
              metrics = ['binary_accuracy'])

## 데이터 제네레이터 정의하기

In [ ]:
batch_size = 32
class_col = ['black', 'blue', 'brown', 'green', 'red', 'white', 
             'dress', 'shirt', 'pants', 'shorts', 'shoes']

# Make Generator
# 2020.08.09 class_mode 수정: other -> raw
# class_mode: ['categorical', ['binary', 'sparse'], ['raw', 'multi_output'], ['input', 'None']]
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df, 
    directory=None,
    x_col = 'image',
    y_col = class_col,
    target_size = (112, 112),
    color_mode='rgb',
    class_mode='raw',
    batch_size=batch_size,
    shuffle = True,
    seed=42
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df, 
    directory=None,
    x_col = 'image',
    y_col = class_col,
    target_size = (112, 112),
    color_mode='rgb',
    class_mode='raw',
    batch_size=batch_size,
    shuffle=True
)

## 제네레이터를 통해 모델 학습시키기

In [ ]:
model.fit(train_generator,
         steps_per_epoch=get_steps(len(train_df), batch_size),
         validation_data = val_generator,
         validation_steps=get_steps(len(val_df), batch_size),
         epochs = 10)

## 테스트 데이터 예측하기

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

# y_col: None, class_mode: None이므로
# test_generator는 image만 반환하고, label은 반환하지 않습니다.
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df, 
    directory=None,
    x_col = 'image',
    y_col = None,
    target_size = (112, 112),
    color_mode='rgb',
    class_mode=None,
    batch_size=batch_size,
    shuffle = False
)

In [ ]:
# 책에는 명시되어 있지 않습니다.
preds = model.predict(test_generator,
                      steps = get_steps(len(test_df), batch_size),
                      verbose = 1)

## 결과 확인하기

In [ ]:
import matplotlib.pyplot as plt
import cv2

# 8개만 예측해보도록 하겠습니다.
do_preds = preds[:8]

for i, pred in enumerate(do_preds):
    plt.subplot(2, 4, i + 1)
    prob = zip(class_col, list(pred))
    # item --> prob
    # contributor: '뱅커'님
    prob = sorted(list(prob), key = lambda z: z[1], reverse = True)[:2]
    
    image = cv2.imread(test_df['image'][i])
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    plt.imshow(image)
    plt.title(f'{prob[0][0]}: {round(prob[0][1] * 100, 2)}% \n {prob[1][0]}: {round(prob[1][1] * 100, 2)}%')
    
plt.tight_layout()
plt.show()